In [2]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/data

!unzip -q '/content/drive/MyDrive/43031/Test/archive.zip' -d '/content/data'


!find /content/data -maxdepth 2 | sed 's/^/    /'


data_dir = '/content/data'




Mounted at /content/drive
    /content/data
    /content/data/Csv
    /content/data/Csv/test.csv
    /content/data/Csv/train.csv
    /content/data/test
    /content/data/test/263
    /content/data/test/71
    /content/data/test/605
    /content/data/test/396
    /content/data/test/88
    /content/data/test/321
    /content/data/test/464
    /content/data/test/159
    /content/data/test/203
    /content/data/test/319
    /content/data/test/619
    /content/data/test/114
    /content/data/test/135
    /content/data/test/755
    /content/data/test/462
    /content/data/test/575
    /content/data/test/422
    /content/data/test/424
    /content/data/test/609
    /content/data/test/348
    /content/data/test/18
    /content/data/test/54
    /content/data/test/651
    /content/data/test/616
    /content/data/test/140
    /content/data/test/65
    /content/data/test/41
    /content/data/test/746
    /content/data/test/155
    /content/data/test/193
    /content/data/test/696
    /content/data

In [3]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


csv_dir     = '/content/data/Csv'
train_csv   = os.path.join(csv_dir, 'train.csv')
test_csv    = os.path.join(csv_dir, 'test.csv')

img_root    = '/content/data'
train_img   = os.path.join(img_root, 'train')
test_img    = os.path.join(img_root, 'test')

print('Train images dirs:', sorted(os.listdir(train_img))[:5], '…')
print('Test  images dirs:', sorted(os.listdir(test_img))[:5], '…')
print('CSV files:', os.listdir(csv_dir))

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_ds = datasets.ImageFolder(train_img, transform=transform)
test_ds  = datasets.ImageFolder(test_img,  transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False)


Train images dirs: ['110', '111', '113', '114', '122'] …
Test  images dirs: ['110', '111', '113', '114', '122'] …
CSV files: ['test.csv', 'train.csv']


In [4]:
from torch.utils.data import random_split, DataLoader

total = len(train_ds)
n_train = int(0.8 * total)
n_val   = total - n_train
train_subset, val_subset = random_split(train_ds, [n_train, n_val])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_subset,   batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds,      batch_size=32, shuffle=False, num_workers=2)

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*28*28,512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512,num_classes)
        )
    def forward(self, x):
        return self.classifier(self.features(x))

from torchvision import models
def get_effnet_b2(fine_tune, num_classes):
    model = models.efficientnet_b2(pretrained=True)
    if not fine_tune:
        for p in model.parameters(): p.requires_grad=False
    else:

        total = len(list(model.parameters()))
        for i, p in enumerate(model.parameters()):
            p.requires_grad = (i/total) >= 0.6
    in_f = model.classifier[1].in_features
    model.classifier = nn.Sequential(nn.Dropout(0.3), nn.Linear(in_f, num_classes))
    return model

import torch.optim as optim

def train_and_eval(model, epochs=5):
    model.to(device)
    opt = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    crit = nn.CrossEntropyLoss()
    best_val = 0
    for _ in range(epochs):
        model.train()
        for x,y in train_loader:
            x,y = x.to(device), y.to(device)
            opt.zero_grad()
            loss = crit(model(x), y)
            loss.backward()
            opt.step()
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x,y in val_loader:
                x,y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                correct += (preds==y).sum().item()
                total += y.size(0)
        best_val = max(best_val, correct/total*100)

    def acc(dl):
        model.eval()
        c = t = 0
        with torch.no_grad():
            for x,y in dl:
                x,y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                c += (preds==y).sum().item()
                t += y.size(0)
        return c/t*100
    return acc(train_loader), best_val, acc(test_loader)

import pandas as pd

num_cls = len(train_ds.dataset.classes) if hasattr(train_ds, 'dataset') else len(train_ds.classes)
results = []
# CNN-1
r1 = train_and_eval(CustomCNN(num_cls))
results.append(['CNN-1 (Custom)', *r1])
# CNN-2
r2 = train_and_eval(get_effnet_b2(fine_tune=False, num_classes=num_cls))
results.append(['CNN-2 (EffNet-B2 Frozen)', *r2])
# CNN-3
r3 = train_and_eval(get_effnet_b2(fine_tune=True,  num_classes=num_cls))
results.append(['CNN-3 (EffNet-B2 FT)', *r3])

df = pd.DataFrame(results, columns=['Model','Train Acc (%)','Val Acc (%)','Test Acc (%)'])
print(df)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 141MB/s]


                      Model  Train Acc (%)  Val Acc (%)  Test Acc (%)
0            CNN-1 (Custom)      42.453101     8.135528      8.062460
1  CNN-2 (EffNet-B2 Frozen)      32.116194    24.693721     23.326960
2      CNN-3 (EffNet-B2 FT)      84.499426    52.526799     49.840663
